In [49]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li pre{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser
`pip install bs4` 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [50]:
# 내 local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청 처리하는 lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [51]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/lecNote/01_python/data/ch14_sample.html')
response

<Response [404]>

In [52]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')    

접근을 못했습니다


In [53]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406 : get, post 오류

404

In [54]:
response.content # 바이너리 형식의 html내용

b"[Errno 2] No such file or directory: '\\\\ai\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [55]:
response.content.decode('utf-8')

"[Errno 2] No such file or directory: '\\\\ai\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [56]:
response.text # HTML파일의 텍스트 내용

"[Errno 2] No such file or directory: '\\\\ai\\\\lecNote\\\\01_python\\\\data\\\\ch14_sample.html'"

In [57]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")
# soup

C:\Users\kimjh\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [58]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

el : None


AttributeError: 'NoneType' object has no attribute 'text'

In [59]:
# soup.select('선택자') : 해당 선택자 모두 
els = soup.select('h1') # h1태그를 list
print('els :', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

els : []
els의 text들 : []
els의 string들 : []
els의 속성들 : []
els의 class 속성들 : []


In [60]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : None
find : None
find : None

select_one : None
find : None


In [61]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : []
find_all : []
find_all : []


In [62]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : []
find_all : []
find_all : []


In [63]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class', 'css'}))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기 ( 네이버증권 -> 시장지표)
- https://finance.naver.com/marketindex/

In [64]:
# 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [65]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")

In [66]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

In [67]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(',') ) ) )
print(prices)

[1464.4, 950.14, 1693.14, 205.55, 154.12, 1.1563, 1.3166, 99.46, 60.13, 1702.93, 4122.0, 194449.28]


In [68]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1464.4, 950.14, 1693.14, 205.55, 154.12, 1.1563, 1.3166, 99.46, 60.13, 1702.93, 4122.0, 194449.28]


In [69]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
        print(t.text, end='\t')
#print([t.text+'\t' for t in titles])

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [70]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [71]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

상승	상승	상승	상승	상승	하락	상승	하락	상승	상승	상승	상승	

In [72]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [73]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,464.40원 - 상승
일본 JPY(100엔) : 950.14원 - 상승
유럽연합 EUR : 1,693.14원 - 상승
중국 CNY : 205.55원 - 상승
달러/일본 엔 : 154.1200엔 - 상승
유로/달러 : 1.1563달러 - 하락
영국 파운드/달러 : 1.3166달러 - 상승
달러인덱스 : 99.4600 - 하락
WTI : 60.13달러 - 상승
휘발유 : 1702.93원 - 상승
국제 금 : 4122.0달러 - 상승
국내 금 : 194449.28원 - 상승


In [74]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,464.40원 -상승
일본 JPY(100엔) : 950.14원 -상승
유럽연합 EUR : 1,693.14원 -상승
중국 CNY : 205.55원 -상승
달러/일본 엔 : 154.1200엔 -상승
유로/달러 : 1.1563달러 -하락
영국 파운드/달러 : 1.3166달러 -상승
달러인덱스 : 99.4600 -하락
WTI : 60.13달러 -상승
휘발유 : 1702.93원 -상승
국제 금 : 4122.0달러 -상승
국내 금 : 194449.28원 -상승


### 2)이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
```
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1, 5, 7, 26, 28, 43]
    보너스 30
```

In [75]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [76]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [77]:
times = soup.select_one('div.win_result strong').text # 1197회
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [78]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title               link
0  한풀 꺾인 비트코인  https://v.daum.net/v/20251110094711892
```

In [79]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 방법2 : 불가
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,비트코인 기부 이어가는 김거석 씨…서울대병원에도 '1비트코인',http://v.daum.net/v/20251111103342643
1,1,"비트코인, ‘셧다운 해제 초읽기’ 10만5000달러…리플 ETF 기대 8% 급등 ...",http://v.daum.net/v/20251111080148418
2,2,"""셧다운 해제 코인 폭발할까""…비트코인 1억5700만원대 회복, 리플은 7%↑",http://v.daum.net/v/20251111094914124
3,3,비트코인 10.6만弗 회복…변동성 우려 여전 [속도내는 원화 스테이블코인],http://v.daum.net/v/20251111112142230
4,4,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
5,5,"""금보다 싸다?""…비트코인 5만6000달러 vs 20만달러 '분분'",http://v.daum.net/v/20251111055113522
6,6,"비트코인 ETF 막힌 한국…서학개미, 미국 레버리지로 '우회 매수' 급증",http://v.daum.net/v/20251111140546725
7,7,또 1비트코인 기부한 78세 김거석 씨...이번엔 서울대병원에,http://v.daum.net/v/20251111144111679
8,8,"서울대병원에 '1 비트코인' 기부한 70대...""시대에 맞는 기부도구""",http://v.daum.net/v/20251111110501306
9,9,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139


In [80]:
items_find_list = [] # 검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])

,번호,제목,링크
0,0,비트코인 기부 이어가는 김거석 씨…서울대병원에도 '1비트코인',http://v.daum.net/v/20251111103342643
1,1,"비트코인, ‘셧다운 해제 초읽기’ 10만5000달러…리플 ETF 기대 8% 급등 ...",http://v.daum.net/v/20251111080148418
2,2,"""셧다운 해제 코인 폭발할까""…비트코인 1억5700만원대 회복, 리플은 7%↑",http://v.daum.net/v/20251111094914124
3,3,비트코인 10.6만弗 회복…변동성 우려 여전 [속도내는 원화 스테이블코인],http://v.daum.net/v/20251111112142230
4,4,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
5,5,"""금보다 싸다?""…비트코인 5만6000달러 vs 20만달러 '분분'",http://v.daum.net/v/20251111055113522
6,6,"비트코인 ETF 막힌 한국…서학개미, 미국 레버리지로 '우회 매수' 급증",http://v.daum.net/v/20251111140546725
7,7,또 1비트코인 기부한 78세 김거석 씨...이번엔 서울대병원에,http://v.daum.net/v/20251111144111679
8,8,"서울대병원에 '1 비트코인' 기부한 70대...""시대에 맞는 기부도구""",http://v.daum.net/v/20251111110501306
9,9,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139


In [81]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,비트코인 기부 이어가는 김거석 씨…서울대병원에도 '1비트코인',http://v.daum.net/v/20251111103342643
1,1,"비트코인, ‘셧다운 해제 초읽기’ 10만5000달러…리플 ETF 기대 8% 급등 ...",http://v.daum.net/v/20251111080148418
2,2,"""셧다운 해제 코인 폭발할까""…비트코인 1억5700만원대 회복, 리플은 7%↑",http://v.daum.net/v/20251111094914124
3,3,비트코인 10.6만弗 회복…변동성 우려 여전 [속도내는 원화 스테이블코인],http://v.daum.net/v/20251111112142230
4,4,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
5,5,"""금보다 싸다?""…비트코인 5만6000달러 vs 20만달러 '분분'",http://v.daum.net/v/20251111055113522
6,6,"비트코인 ETF 막힌 한국…서학개미, 미국 레버리지로 '우회 매수' 급증",http://v.daum.net/v/20251111140546725
7,7,또 1비트코인 기부한 78세 김거석 씨...이번엔 서울대병원에,http://v.daum.net/v/20251111144111679
8,8,"서울대병원에 '1 비트코인' 기부한 70대...""시대에 맞는 기부도구""",http://v.daum.net/v/20251111110501306
9,9,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139


In [82]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [83]:
collect_news_list('AI', 1)

[[0,
  ' AI로 안 되는 게 없네…"오픈AI, 헬스케어 분야 진출 검토" ',
  'http://v.daum.net/v/20251111042100588'],
 [1, " AI 거품론에도… 데이터센터 투자 '쑥쑥' ", 'http://v.daum.net/v/20251111041609575'],
 [2,
  ' 야놀자, 첫 AI 해커톤 개최···이수진 창업자 "AI 혁신, 전 그룹 문화로 확산" ',
  'http://v.daum.net/v/20251110205114489'],
 [3,
  ' [부자들의 투자노트] LG AI연구원 이문태 "AI 거품 가능성 낮아" ',
  'http://v.daum.net/v/20251111074522130'],
 [4,
  ' [리셋 코리아] AI 대전환기, 소버린AI 도전 피하지 말아야 ',
  'http://v.daum.net/v/20251110002138457'],
 [5,
  ' “AI·인간 \'공동과학자\' 시대…R&D 속도, AI 활용 능력이 가른다" ',
  'http://v.daum.net/v/20251110050654976'],
 [6,
  " 초중고 정보 교과 내 AI 교육 확대…모든 교육청 'AI 지원센터' 설립 ",
  'http://v.daum.net/v/20251110140139274'],
 [7,
  ' 초중고 ‘전 생애 AI 교육’…‘AI 박사’ 5년 반 만에 키운다 ',
  'http://v.daum.net/v/20251110205314501'],
 [8, ' 인텔 AI 사업 총괄 CTO, 오픈AI행 ', 'http://v.daum.net/v/20251111132649261'],
 [9,
  ' 한국 AI 신뢰 투자, 글로벌 평균 절반…"AI 거버넌스 미비" ',
  'http://v.daum.net/v/20251111102213056']]

In [84]:
import time
keywords = ['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
===2번째 검색어 동대문 검색 결과 수집 중입니다===


In [85]:
result0_df = pd.DataFrame(result0, columns=['no', 'title','link'])
result1_df = pd.DataFrame(result1, columns=['no', 'title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 ‘핫플’ 됐다",http://v.daum.net/v/20251111111251709
2,2,"동대문구, 4년간 미취업 청년 4935명 자격취득 지원[동네방네]",http://v.daum.net/v/20251111105649790
3,3,"동대문구, 5대 의료단체와 손잡고 ‘통합돌봄체계’ 구축 나선다",http://v.daum.net/v/20251111110225154
4,4,"동대문구, 4년간 청년 4935명에 자격증 응시료 8억 원 지원",http://v.daum.net/v/20251111075113226


In [86]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False, encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라어저를 통해 요청하는 것 처럼 보이기

In [87]:
# 방법2 : 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
# headers = {'User-Agent':
#           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent', 
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')

response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


- https://www.melon.com/chart/index.htm
    * https://wwww.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [88]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [89]:
# 방법2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
# soup

In [90]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
5위 | Drowning - WOODZWOODZ
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
10위 | Soda Pop - KPop Demon Hunters C
11위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
12위 | FAMOUS - ALLDAY PROJECTALLDAY
13위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | IRIS OUT - Kenshi YonezuKenshi 
16위 | Rich Man - aespaaespa
17위 | 한번 더 이별 - 이창섭이창섭
18위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
19위 | 너에게 닿기를 - 10CM10CM
20위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
21위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
22위 | body - 다영 (DAYOUNG)다영 (DAYO
23위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | 천상연 - 이창섭이창섭
27위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU (악뮤)AKMU (악뮤)
28위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
29위 |

### 5) 네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수만큼

In [91]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
# url = 'https://kin.naver.com/search/list.naver'
# params = {'query' : keyword}
# response = get(url, params=params)
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
response = get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
# soup

200


In [92]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://kin.naver.com/search/list.naver?query=%EC%B3%87%EC%A7%80%ED%94%BC%ED%8B%B0
200


In [93]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 크롤링한 데이터를 담을 list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = urlopen(url)
    # print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link  = item.attrs.get('href')
        items_list.append({
            'title': title,
            'link' : link
        })
df = pd.DataFrame(items_list)
print(df.shape)
print(df.loc[29, 'link'])
df.sample()

(30, 2)
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31301&docId=485933293&qb=7LOH7KeA7ZS87Yuw&enc=utf8


,title,link
14,쳇지피티 안됨,https://kin.naver.com/qna/detail.naver?d1id=1&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색 (open API 사용 O)
- 네이버 개발자센터에서 애플리케이션등록(이름/검색/WEB설정http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv`
- 특정 keyword를 지식검색(데이터수 30개)

In [94]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [95]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    #dotenv_path='.env'
           ) # 현 소스와 같은 폴더내의 .env를 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [96]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    print(response_body.decode('utf-8')[:50])
else:
    print("Error Code:" + rescode)
items = json.loads(response_body.decode('utf-8'))['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

<class 'str'>
{
	"lastBuildDate":"Tue, 11 Nov 2025 14:57:46 +090


,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


In [97]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '쳇지피티'
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=30'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link  = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
제목,링크,썸네일,sizeheight,sizewidth
```

In [98]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
19,옷자락 여성 와이드핏 워싱진 뒷밴딩 다트 일자청바지 : 옷자락 OSJARAK,https://shop-phinf.pstatic.net/20251002_20/175...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [99]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
df.to_csv('data/ch14_청바지.csv')
df.sample()

,제목,링크,썸네일,sizeheight,sizewidth
15,썬더볼츠 남성 남친 데님바지 일자핏 청바지 여름 루즈핏 일자 팬츠 비즈니스 하이웨스...,https://shop-phinf.pstatic.net/20250211_189/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [100]:
print(df.loc[25, '링크'])
print(df.loc[25, '썸네일'])

https://image.msscdn.net/thumbnails/images/goods_img/20251002/5566221/5566221_17593796923059_big.jpg?w=780
https://search.pstatic.net/sunny/?type=b150&src=https://image.msscdn.net/thumbnails/images/goods_img/20251002/5566221/5566221_17593796923059_big.jpg?w=780


In [101]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import requests
    file_extension = link.split('.')[-1] # 확장자
    quote_index = file_extension.find('?') # 확장자뒤에 ?가 있는 위치 .jpg?w=780
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content # 바이너리
    with open(f'image/{attr}_{idx+1:02}_{query}.{file_extension}', 'wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[25, '링크'], '청바지')

In [102]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

= = = 20% 진행중 = = =
= = = 40% 진행중 = = =
= = = 60% 진행중 = = =
= = = 80% 진행중 = = =
= = = 완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
70,1 1 2개 아동 여아 데님 부츠컷 청바지 2종세트 데님 2종세트 130,http://shopping.phinf.naver.net/main_5677535/5...,https://search.pstatic.net/common/?type=b150&s...,780,780


## 2.3 XML 웹데이터 수집
 - RSS / open API 를 통한 XML 웹데이터
 
 ### 1. 전국 날씨 RSS를 BeautifulSoup을 이용한 크롤링
 

In [117]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
#방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, 'xml')

#방법2
target = urlopen(url)
soup = BeautifulSoup(target,'xml')

local_tas = soup.select('local_ta')
len(local_tas)
for local_ta in local_tas:
    local_name=local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text 
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text 
    
    items_list.append({
        '지역': local_name,
        '1주평년기온' : week1_normalYear,
        '1주기온범위' : week1_similarRange,
        '1주낮을확률' : week1_minVal,
        '1주비슷할확률' : week1_similarVal,
        '1주높을확률' : week1_maxVal,
    })
df = pd.DataFrame(items_list)

In [123]:
df['1주평년기온'] = df['1주평년기온'] .astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'] .astype(np.float64)
df['1주비슷할확률'] = df['1주비슷할확률'] .astype(np.float64)
df['1주높을확률'] = df['1주높을확률'] .astype(np.float64)
df.head()

,지역,1주평년기온,1주기온범위,1주낮을확률,1주비슷할확률,1주높을확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20.0,40.0,40.0
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20.0,40.0,40.0
2,강원도 영서,2.6,1.5~3.7,20.0,40.0,40.0
3,강원도 영동,5.8,4.9~6.7,20.0,40.0,40.0
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20.0,40.0,40.0


## 2. XML 응답하는 open API 활용
 - data.go.kr에서
     - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과정류장ID)
     - 서울특별시_버스위치정보조회 서비스(실시간 버스위치 목록)

In [ ]:
# STEP1 버스 번호의 버스 ID 받아오기
# 서울특별시_노선정보조회 서비스 -3번 기능 (getBusRouteList)이용

In [23]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))
# print(os.getenv('KEY1'))

True

In [29]:
from bs4 import BeautifulSoup
#urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum = quote('종로07')
busNum = '273'
key = os.getenv('KEY')
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
savefilename1 = 'Data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) #xml 파일(url)을 savefilename으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml,'xml')

In [48]:
import requests
from urllib.parse import quote
busNum = '162'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum: 
        busRouteId = item.select_one('busRouteId').text
        print(busRouteId)
        break;
print('busRouteId =', busRouteId)

100100034
busRouteId = 100100034


In [49]:
#step2 bus id로 버스 정류장 목록 받아오기 (정류장명, 정류장 id, 경도, 위도)
#서울특별시노선정보조회서비스 -4번기능 (getStationByRoutelist)

In [51]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
print(url2)
response = requests.get(url2)
soup = BeautifulSoup(response.content,'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text #정류소 이름
    station  =  itemList.find('station').text #정류소 ID
    gpsX    = itemList.find('gpsX').text #경도
    gpsY = itemList.find('gpsY').text
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명', 'id','경도','위도'])
df_station.head()

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=f1c4bb977979e323d4d7dbd2106e333c762552f43a942bc8cbf55b4ada1dd35a&busRouteId=100100034
162번 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [ ]:
#step3 bus id로 실시간 위치정보 받아오기 (정류장 id)
#서울특별시_버스위치정보조회서비스 2번(getBusposByRtidList)

In [61]:
url3=f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text,'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대 입니다')

bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text #차량번호
    congetion = itemList.select_one('congetion').text #혼잡도
    #0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion == '0'\
            else '여유' if congetion =='3' \
            else '보통' if congetion =='4'\
            else '혼잡' if congetion =='5'\
            else '매우혼잡'
    bus_position.append({
        '차량번호' : plainNo,
        '혼잡도' : congetion
    })
df_position = pd.DataFrame(bus_potision)
df_position
    

운행중인 버스는 22대 입니다


""
